In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install shap
!pip install catboost
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostRegressor, Pool, CatBoostClassifier, cv
os.chdir('/content/drive/My Drive/ML')

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.1/540.1 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 7.4 MB/s eta 0:00:00


In [ ]:
df = pd.read_csv('features.csv')
df.drop(['interior_color', 'legroom', 'combined_fuel_economy', 'body_type', 'size_of_vehicle'], axis = 1, inplace = True)
df.head(2)

,vin,engine_displacement,horsepower,make_name,mileage,price,transmission_display,trimId,year
0,3C4NJBB0LT255912,2400.0,180.0,Jeep,9.0,28620.0,9-Speed Automatic,t87047,2020
1,1GTV2EC5GZ384010,6200.0,420.0,GMC,73112.0,31690.0,Automatic,t60025,2016


In [ ]:
features = list(df)
features.remove('price')
features

['vin',
 'engine_displacement',
 'horsepower',
 'make_name',
 'mileage',
 'transmission_display',
 'trimId',
 'year']

In [ ]:
df_notna = df[[*features, 'price']].dropna()
SEED = 42
X = df_notna[features]
y = df_notna.price
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.66, random_state=SEED)

In [ ]:
categorical_features_indices = np.where(X.dtypes != float)[0]
train_pool = Pool(X_train, y_train, cat_features=categorical_features_indices)
test_pool = Pool(X_test, y_test, cat_features=categorical_features_indices)

In [ ]:
cat = CatBoostRegressor(iterations = 100).fit(train_pool, eval_set = test_pool)

Learning rate set to 0.5
0:	learn: 9961.3259575	test: 9758.2017164	best: 9758.2017164 (0)	total: 503ms	remaining: 49.8s
1:	learn: 7505.6862867	test: 7238.2697395	best: 7238.2697395 (1)	total: 946ms	remaining: 46.4s
2:	learn: 6395.9380506	test: 6072.4936451	best: 6072.4936451 (2)	total: 1.38s	remaining: 44.6s
3:	learn: 5881.2522584	test: 5538.1151112	best: 5538.1151112 (3)	total: 1.71s	remaining: 41s
4:	learn: 5664.5415304	test: 5317.5545214	best: 5317.5545214 (4)	total: 2.02s	remaining: 38.4s
5:	learn: 5504.8692683	test: 5149.6152821	best: 5149.6152821 (5)	total: 2.33s	remaining: 36.5s
6:	learn: 5412.1143451	test: 5057.7653444	best: 5057.7653444 (6)	total: 2.65s	remaining: 35.2s
7:	learn: 5329.8007942	test: 4981.0386682	best: 4981.0386682 (7)	total: 2.95s	remaining: 34s
8:	learn: 5256.8887697	test: 4902.2970832	best: 4902.2970832 (8)	total: 3.29s	remaining: 33.3s
9:	learn: 5208.7420565	test: 4859.8672619	best: 4859.8672619 (9)	total: 3.62s	remaining: 32.6s
10:	learn: 5164.1528568	test:

In [ ]:
del df_notna

In [ ]:
from sklearn.metrics import mean_squared_error
y_pred = cat.predict(X_test)
print('Naive:', mean_squared_error(y_test, y_test - y_test.mean(), squared=False))
print('RMSE:', mean_squared_error(y_test, y_pred, squared=False))

Naive: 27773.15445523579
RMSE: 3903.3344807358667


In [ ]:
df_test = pd.read_csv('test.csv', usecols=['vin','engine_displacement','horsepower','make_name','mileage','transmission_display','trimId','year',])
df_submission = pd.read_csv('submission.csv')
df_test.info()

In [ ]:
df_test['engine_displacement'] = df_test.engine_displacement.fillna(df_test.engine_displacement.mean())
df_test['horsepower'] = df_test.horsepower.fillna(df_test.horsepower.mean())
df_test['transmission_display'] = df_test.transmission_display.fillna('Automatic')
df_test['trimId'] = df_test.trimId.fillna('t87739')
df_test['mileage'] = df_test.mileage.fillna(df_test.mileage.mean())

In [ ]:
df_test['price'] = cat.predict(df_test[features])
df_test[['vin', 'price']].to_csv('submission.csv', index=False)
!zip submission.zip submission.csv

updating: submission.csv (deflated 56%)


In [ ]:
#explainer = shap.TreeExplainer(cat)
#shap_values = explainer.shap_values(X)
#shap.summary_plot(shap_values, X)